<a href="https://colab.research.google.com/github/pawel0508/NeuralNetworks/blob/main/Regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision = 12, suppress = True, linewidth = 150)
pd.options.display.float_format = "{:.6f}".format
tf.__version__


'2.3.0'

**Ładowanie danych**

In [2]:
raw_data = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
data_set = raw_data.copy()
data_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [4]:
data_set.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


In [8]:
data_set.isnull().sum()/len(data_set)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.010029
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [10]:
data_set.dropna(inplace=True)

In [11]:
data_set.isnull().sum()/len(data_set)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.000000
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [12]:
data_set.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [17]:
data_set.describe(include = ['object'])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [18]:
data_set.ocean_proximity.value_counts()

<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: ocean_proximity, dtype: int64

In [24]:
px.histogram(data_set, x = 'median_house_value', template = 'simple_white')

In [25]:
data_set.median_house_value.value_counts()

500001.000000    958
137500.000000    119
162500.000000    116
112500.000000    103
187500.000000     92
                ... 
420500.000000      1
419000.000000      1
443000.000000      1
343600.000000      1
319500.000000      1
Name: median_house_value, Length: 3833, dtype: int64

In [27]:
index_to_drop = data_set[data_set['median_house_value'] == 500001].index

In [29]:
dataset = data_set.drop(index_to_drop)

In [31]:
px.histogram(dataset, x = 'median_house_value', template = 'simple_white')

In [33]:
dummies_dataset = pd.get_dummies(dataset, drop_first = True)

In [35]:
dummies_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,0,0,1,0
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,0,0,1,0
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,0,0,1,0
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,0,0,1,0
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,0,0,1,0


In [36]:
train_dataset = dummies_dataset.sample(frac = 0.8, random_state=0)
test_dataset = dummies_dataset.drop(train_dataset.index)

In [37]:
print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


In [38]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122.690000,38.510000,18.000000,3364.000000,501.000000,1442.000000,506.000000,6.685400,313000.000000,0,0,0,0
1859,-124.140000,41.950000,21.000000,2696.000000,578.000000,1208.000000,494.000000,2.275000,122400.000000,0,0,0,1
51,-122.270000,37.820000,43.000000,1868.000000,456.000000,1061.000000,407.000000,1.504500,93800.000000,0,0,1,0
11192,-117.930000,33.820000,28.000000,2444.000000,555.000000,1848.000000,567.000000,3.017900,198800.000000,0,0,0,0
20355,-118.960000,34.190000,16.000000,1807.000000,346.000000,587.000000,296.000000,1.981100,162500.000000,0,0,0,0


In [39]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260000,37.850000,52.000000,2202.000000,434.000000,910.000000,402.000000,3.203100,281500.000000,0,0,1,0
13,-122.260000,37.840000,52.000000,696.000000,191.000000,345.000000,174.000000,2.673600,191300.000000,0,0,1,0
19,-122.270000,37.840000,52.000000,1503.000000,298.000000,690.000000,275.000000,2.603300,162900.000000,0,0,1,0
26,-122.280000,37.850000,49.000000,1130.000000,244.000000,607.000000,239.000000,2.459700,93800.000000,0,0,1,0
40,-122.260000,37.830000,52.000000,1665.000000,419.000000,946.000000,395.000000,2.097800,155400.000000,0,0,1,0


In [40]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], 
                  color='median_house_value', height=700, template = 'simple_white')

In [43]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')

train_stats = train_stats.transpose()


In [44]:
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000000,-119.558780,2.006237,-124.350000,-121.750000,-118.490000,-117.990000,-114.470000
latitude,15580.000000,35.648614,2.147016,32.550000,33.930000,34.270000,37.730000,41.950000
housing_median_age,15580.000000,28.489217,12.505895,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,15580.000000,2620.014506,2195.654212,2.000000,1441.000000,2112.000000,3119.000000,39320.000000
total_bedrooms,15580.000000,539.900578,424.499548,2.000000,299.000000,436.000000,647.000000,6445.000000
population,15580.000000,1441.193068,1160.542775,3.000000,801.000000,1179.000000,1746.000000,35682.000000
households,15580.000000,501.099487,385.039835,2.000000,283.000000,411.000000,605.000000,6082.000000
median_income,15580.000000,3.672427,1.570297,0.499900,2.519975,3.440500,4.581425,15.000100
ocean_proximity_INLAND,15580.000000,0.333569,0.471503,0.000000,0.000000,0.000000,1.000000,1.000000
ocean_proximity_ISLAND,15580.000000,0.000193,0.013876,0.000000,0.000000,0.000000,0.000000,1.000000


In [45]:
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

In [47]:
def norm(x):
  return (x-train_stats['mean'])/train_stats['std']

In [48]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [50]:
normed_train_data.isnull().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64

In [51]:
normed_test_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.346412,1.025324,1.879976,-0.190383,-0.249472,-0.457711,-0.257375,-0.298878,-0.707459,-0.013877,2.888874,-0.378230
13,-1.346412,1.020666,1.879976,-0.876283,-0.821910,-0.944552,-0.849521,-0.636075,-0.707459,-0.013877,2.888874,-0.378230
19,-1.351396,1.020666,1.879976,-0.508739,-0.569849,-0.647277,-0.587211,-0.680844,-0.707459,-0.013877,2.888874,-0.378230
26,-1.356380,1.025324,1.640089,-0.678620,-0.697057,-0.718796,-0.680707,-0.772292,-0.707459,-0.013877,2.888874,-0.378230
40,-1.346412,1.016008,1.879976,-0.434957,-0.284807,-0.426691,-0.275555,-1.002758,-0.707459,-0.013877,2.888874,-0.378230


In [53]:
normed_train_data = normed_train_data.values
normed_test_data = normed_test_data.values

**Budowa modelu**

In [57]:
model = Sequential()
model.add(Dense(1024, activation = 'relu', kernel_regularizer= 'l2', input_shape = [len(train_dataset.keys())]))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1)) # warstwa wyjściowa bez zmiany funkcji aktywacji (czyli f. aktywacji liniowa, bo mamy regresje)

model.compile(optimizer = 'adam',
              loss = 'mse',
              metrics = ['mse', 'mae'])

In [58]:
model.fit(x = normed_train_data, y = train_labels, epochs = 150, batch_size = 32, validation_split = 0.2, verbose = 1) 

Epoch 1/150
390/390 [==============================] - 4s 11ms/step - loss: 16879724544.0000 - mse: 16879724544.0000 - mae: 94397.7812 - val_loss: 4577250816.0000 - val_mse: 4577250816.0000 - val_mae: 48783.3750
Epoch 2/150
390/390 [==============================] - 4s 11ms/step - loss: 4148434176.0000 - mse: 4148434176.0000 - mae: 46312.0117 - val_loss: 3779568384.0000 - val_mse: 3779568384.0000 - val_mae: 44888.9844
Epoch 3/150
390/390 [==============================] - 4s 11ms/step - loss: 3745354240.0000 - mse: 3745354240.0000 - mae: 43968.7812 - val_loss: 3598964480.0000 - val_mse: 3598964480.0000 - val_mae: 43237.7344
Epoch 4/150
390/390 [==============================] - 4s 10ms/step - loss: 3613089792.0000 - mse: 3613089792.0000 - mae: 43084.4102 - val_loss: 3503202816.0000 - val_mse: 3503202816.0000 - val_mae: 42845.4414
Epoch 5/150
390/390 [==============================] - 4s 10ms/step - loss: 3548996608.0000 - mse: 3548996608.0000 - mae: 42656.0938 - val_loss: 3475223040.00

In [59]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
  print(f'{name:8}{value:.4f}')

122/122 [==============================] - 0s 3ms/step - loss: 2361929728.0000 - mse: 2361929728.0000 - mae: 33900.8633
loss    2361929728.0000
mse     2361929728.0000
mae     33900.8633


In [61]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

array([242014.03 , 194039.86 , 179359.4  , ..., 112265.01 , 154375.98 ,  89551.375], dtype=float32)

In [62]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000000,242014.031250
13,191300.000000,194039.859375
19,162900.000000,179359.406250
26,93800.000000,152846.828125
40,155400.000000,185256.562500


In [73]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines', line = dict(color ='red')))
fig.update_layout(template = 'simple_white')
fig.show()

In [74]:
pred.head()

,median_house_value,predictions
10,281500.000000,242014.031250
13,191300.000000,194039.859375
19,162900.000000,179359.406250
26,93800.000000,152846.828125
40,155400.000000,185256.562500


In [75]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000000,242014.031250,39485.968750
13,191300.000000,194039.859375,-2739.859375
19,162900.000000,179359.406250,-16459.406250
26,93800.000000,152846.828125,-59046.828125
40,155400.000000,185256.562500,-29856.562500
